In [1]:
# %load_ext autoreload
# %autoreload 2

# %matplotlib inline

In [2]:
#export
from exp.nb_11 import *

# Serializing the model

In [3]:
path = untar_data(URLs.IMAGEWOOF_160)

In [4]:
size = 128
bs = 64

tfms = [make_rgb, RandomResizedCrop(size, scale=(0.35,1)), np_to_float, PilRandomFlip()]
val_tfms = [make_rgb, CenterCrop(size), np_to_float]

il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())

ll.valid.x.tfms = val_tfms
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=8)

In [5]:
len(il)

12954

In [6]:
loss_func = LabelSmoothingCrossEntropy()
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)

In [7]:
def sched_1cycle(lr, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr), ParamScheduler('mom', sched_mom)]

In [8]:
lr = 3e-3
pct_start = 0.5
cbsched = sched_1cycle(lr, pct_start)

In [9]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [10]:
# learn.fit(40, cbsched)

In [11]:
st = learn.model.state_dict()

In [12]:
type(st)

collections.OrderedDict

In [13]:
', '.join(st.keys())

'0.0.weight, 0.1.weight, 0.1.bias, 0.1.running_mean, 0.1.running_var, 0.1.num_batches_tracked, 1.0.weight, 1.1.weight, 1.1.bias, 1.1.running_mean, 1.1.running_var, 1.1.num_batches_tracked, 2.0.weight, 2.1.weight, 2.1.bias, 2.1.running_mean, 2.1.running_var, 2.1.num_batches_tracked, 4.0.convs.0.0.weight, 4.0.convs.0.1.weight, 4.0.convs.0.1.bias, 4.0.convs.0.1.running_mean, 4.0.convs.0.1.running_var, 4.0.convs.0.1.num_batches_tracked, 4.0.convs.1.0.weight, 4.0.convs.1.1.weight, 4.0.convs.1.1.bias, 4.0.convs.1.1.running_mean, 4.0.convs.1.1.running_var, 4.0.convs.1.1.num_batches_tracked, 4.1.convs.0.0.weight, 4.1.convs.0.1.weight, 4.1.convs.0.1.bias, 4.1.convs.0.1.running_mean, 4.1.convs.0.1.running_var, 4.1.convs.0.1.num_batches_tracked, 4.1.convs.1.0.weight, 4.1.convs.1.1.weight, 4.1.convs.1.1.bias, 4.1.convs.1.1.running_mean, 4.1.convs.1.1.running_var, 4.1.convs.1.1.num_batches_tracked, 5.0.convs.0.0.weight, 5.0.convs.0.1.weight, 5.0.convs.0.1.bias, 5.0.convs.0.1.running_mean, 5.0.convs

In [14]:
st['10.bias']

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [15]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)

In [16]:
# torch.save(st, mdl_path/'iw5')

# Pets

In [17]:
pets = untar_data(URLs.PETS)

In [18]:
pets.ls()

[Path('/home/sandmann/.fastai/data/oxford-iiit-pet/small-256'),
 Path('/home/sandmann/.fastai/data/oxford-iiit-pet/image_gen'),
 Path('/home/sandmann/.fastai/data/oxford-iiit-pet/crappy'),
 Path('/home/sandmann/.fastai/data/oxford-iiit-pet/annotations'),
 Path('/home/sandmann/.fastai/data/oxford-iiit-pet/models'),
 Path('/home/sandmann/.fastai/data/oxford-iiit-pet/small-96'),
 Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images')]

In [19]:
pets_path = pets/'images'

In [20]:
il = ImageList.from_files(pets_path, tfms=tfms)

In [21]:
il

ImageList (7390 items)
[Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/Birman_115.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/leonberger_142.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/Bombay_68.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/japanese_chin_26.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/saint_bernard_149.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/Ragdoll_41.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/japanese_chin_32.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/Ragdoll_68.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/Persian_202.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/scottish_terrier_143.jpg')...]
Path: /home/sandmann/.fastai/data/oxford-iiit-pet/images

In [22]:
#export
def random_splitter(fn, p_valid): return random.random() < p_valid

In [23]:
random.seed(42)

In [24]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [25]:
sd

SplitData
Train: ImageList (6667 items)
[Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/Birman_115.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/Bombay_68.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/japanese_chin_26.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/saint_bernard_149.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/Ragdoll_41.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/japanese_chin_32.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/Persian_202.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/english_setter_157.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/Bombay_215.jpg'), Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/Bengal_69.jpg')...]
Path: /home/sandmann/.fastai/data/oxford-iiit-pet/images
Valid: ImageList (723 items)
[Path('/home/sandmann/.fastai/data/oxford-iiit-pet/images/leonberger_142.jpg'), Path('/home/sa

In [26]:
n = il.items[0].name
n

'Birman_115.jpg'

In [27]:
re.findall(r'^(.*)_\d+\.jpg$', n)[0]

'Birman'

In [28]:
def pet_labeler(fn): return re.findall(r'^(.*)_\d+\.jpg$', fn.name)[0]

In [29]:
proc = CategoryProcessor()

In [30]:
ll = label_by_func(sd, pet_labeler, proc_y=proc)

In [31]:
', '.join(proc.vocab)

'Birman, Bombay, japanese_chin, saint_bernard, Ragdoll, Persian, english_setter, Bengal, great_pyrenees, basset_hound, Egyptian_Mau, american_bulldog, english_cocker_spaniel, newfoundland, american_pit_bull_terrier, samoyed, staffordshire_bull_terrier, shiba_inu, german_shorthaired, miniature_pinscher, leonberger, yorkshire_terrier, scottish_terrier, Sphynx, havanese, beagle, keeshond, boxer, British_Shorthair, pomeranian, Abyssinian, Maine_Coon, pug, wheaten_terrier, Russian_Blue, chihuahua, Siamese'

In [32]:
ll.valid.x.tfms = val_tfms

In [33]:
c_out = len(proc.vocab)

In [34]:
data = ll.to_databunch(bs, c_in=3, c_out=c_out, num_workers=8)

In [35]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [36]:
# learn.fit(5, cbsched)

# Custom head

In [37]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [38]:
st = torch.load(mdl_path/'iw5')

In [39]:
m = learn.model

In [40]:
m.load_state_dict(st)

<All keys matched successfully>

In [41]:
cut = [i for i,o in enumerate(m.children()) if isinstance(o, nn.AdaptiveAvgPool2d)][0]
m_cut = m[:cut]

In [42]:
xb,yb = get_batch(data.valid_dl, learn)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [43]:
pred = m_cut(xb)

/home/sandmann/anaconda3/envs/fastbook/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [44]:
pred.shape

torch.Size([128, 512, 4, 4])

In [45]:
ni = pred.shape[1]

In [46]:
#export
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=1):
        super().__init__()
        self.output_size = sz
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
        
    def forward(self, x): return torch.cat([self.ap(x), self.mp(x)], 1)

In [47]:
nh = 40

m_new = nn.Sequential(m_cut,
                     AdaptiveConcatPool2d(), Flatten(),
                     nn.Linear(2*ni, data.c_out))

In [48]:
learn.model = m_new.cuda()

In [49]:
# learn.fit(5, cbsched)

# adapt_model and gradual unfreezing

In [50]:
def adapt_model(learn, data):
    cut = [i for i,o in enumerate(learn.model.children()) if isinstance(o, nn.AdaptiveAvgPool2d)][0]
    m_cut = learn.model[:cut]
    xb,yb = get_batch(data.valid_dl, learn)
    pred = m_cut(xb)
    ni = pred.shape[1]
    
    m_new = nn.Sequential(m_cut,
                         AdaptiveConcatPool2d(), Flatten(),
                         nn.Linear(2*ni, data.c_out))
    learn.model = m_new.to(xb.device)

In [51]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))

<All keys matched successfully>

In [52]:
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [53]:
for p in learn.model[0].parameters(): p.requires_grad_(False)

In [54]:
# learn.fit(3, sched_1cycle(1e-2, 0.5))

In [55]:
for p in learn.model[0].parameters(): p.requires_grad_(True)

In [56]:
# learn.fit(5, cbsched, reset_opt=True)

# Batch Norm Transfer

In [57]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [58]:
def apply_mod(m, f):
    f(m)
    for l in m.children(): apply_mod(l, f)
    
def set_grad(m, b):
    if isinstance(m, (nn.Linear, nn.BatchNorm2d)): return
    if hasattr(m, 'weight'):
        for p in m.parameters(): p.requires_grad_(b)

In [59]:
apply_mod(learn.model, partial(set_grad, b=False))

In [60]:
# learn.fit(3, sched_1cycle(1e-2, 0.5))

In [61]:
apply_mod(learn.model, partial(set_grad, b=True))

In [62]:
# learn.fit(5, cbsched, reset_opt=True)

In [63]:
learn.model.apply(partial(set_grad, b=False));

# Discriminative LR and param groups

In [64]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [65]:
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [66]:
def bn_splitter(m):
    def _bn_splitter(l, g1, g2):
        if isinstance(l, nn.BatchNorm2d): g2 += l.parameters()
        elif hasattr(l, 'weight'): g1 += l.parameters()
        for ll in l.children(): _bn_splitter(ll, g1, g2)
        
    g1,g2 = [],[]
    _bn_splitter(m[0], g1, g2)
    g2 += m[1:].parameters()

    return g1,g2

In [67]:
a,b = bn_splitter(learn.model)

In [68]:
test_eq(len(a)+len(b), len(list(learn.model.parameters())))

In [69]:
Learner.ALL_CBS

{'after_backward',
 'after_batch',
 'after_cancel_batch',
 'after_cancel_epoch',
 'after_cancel_train',
 'after_epoch',
 'after_fit',
 'after_loss',
 'after_pred',
 'after_step',
 'begin_batch',
 'begin_epoch',
 'begin_fit',
 'begin_validate'}

In [70]:
#export
from types import SimpleNamespace
cb_types = SimpleNamespace(**{o:o for o in Learner.ALL_CBS})

In [71]:
#export
class DebugCallback(Callback):
    _order = 999
    def __init__(self, cb_name, f=None): self.cb_name,self.f = cb_name,f
    def __call__(self, cb_name):
        if cb_name == self.cb_name:
            if self.f: self.f(self.run)
            else: set_trace()

In [72]:
def _print_det(o):
    print(len(o.opt.param_groups), o.opt.hypers, sep='\n')
    raise CancelTrainException()

In [73]:
#export
def sched_1cycle(lrs, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    lrs = listify(lrs)
    phases = create_phases(pct_start)
    sched_lr = [combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5)) for lr in lrs]
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    
    return [ParamScheduler('lr', sched_lr), ParamScheduler('mom', sched_mom)]

In [74]:
disc_lr_sched = sched_1cycle([0,3e-2], 0.5)

In [75]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette, splitter=bn_splitter)

learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [76]:
learn.fit(1, disc_lr_sched + [DebugCallback(cb_types.after_batch, _print_det)])

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


2
[{'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0, 'sqr_mom': 0.99}, {'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0030000000000000512, 'sqr_mom': 0.99}]


In [77]:
# learn.fit(3, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.545230,0.362082,2.300637,0.450899,00:19
1,2.246772,0.457627,2.307571,0.439834,00:18
2,1.970470,0.545973,1.899560,0.576764,00:18


In [78]:
disc_lr_sched = sched_1cycle([1e-3,1e-2], 0.3)

In [79]:
# learn.fit(5, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.817210,0.603720,1.872048,0.572614,00:18
1,1.836615,0.590970,2.411371,0.410788,00:18
2,1.694964,0.650068,1.812372,0.604426,00:18
3,1.545127,0.702415,1.682303,0.654219,00:21
4,1.453181,0.740363,1.610572,0.688797,00:20


In [81]:
!python notebook2script.py 11a_transfer_learning.ipynb

Converted 11a_transfer_learning.ipynb to exp/nb_11a.py
